In [30]:
import pkuseg
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

seg = pkuseg.pkuseg()

In [31]:
article_names = os.listdir('data')

In [32]:
article_contents = list()
for article_name in article_names:
    file_name = 'data/' + article_name
    file = open(file_name, 'r')
    article_content = file.read()
    file.close()
    article_contents.append(article_content)

In [46]:
def get_articles_from_folder(foldername: str) -> [str]:
    article_names = os.listdir(foldername)
    article_contents = list()
    for article_name in article_names:
        file_name = 'data/' + article_name
        file = open(file_name, 'r')
        article_content = file.read()
        file.close()
        article_contents.append(article_content)
    
    return article_contents

In [49]:
x = get_articles_from_folder('data')

In [33]:
article_words = list()
for i in tqdm(range(len(article_contents))):
    article_content = article_contents[i]
    words = seg.cut(article_content)
    article_words.append(words)

100%|██████████| 73/73 [00:18<00:00,  3.93it/s]


In [34]:
def counter(l: list) -> dict:
    stats = dict()
    for i in range(len(l)):
        item = l[i]
        if item not in stats:
            stats[item] = 1
        else:
            stats[item] = stats[item] + 1
    
    return stats

In [35]:
article_stats = list()
for i in tqdm(range(len(article_words))):
    article_stats.append(counter(article_words[i]))

100%|██████████| 73/73 [00:00<00:00, 714.13it/s]


In [36]:
all_term_stats = dict()
for i in tqdm(range(len(article_stats))):
    article_stat = article_stats[i]
    for word, count in article_stat.items():
        if word not in all_term_stats:
            all_term_stats[word] = count
        else:
            all_term_stats[word] = all_term_stats[word] + count

100%|██████████| 73/73 [00:00<00:00, 2145.72it/s]


In [37]:
all_term_vector = list(all_term_stats.keys())

In [38]:
doc_matrix = list()
for i in tqdm(range(len(article_stats))):
    stat_of_this_article = article_stats[i]
    doc_matrix_row = list()
    for term in all_term_vector:
        if term in stat_of_this_article:
            count = stat_of_this_article[term]
        else:
            count = 0
        doc_matrix_row.append(count)
    
    doc_matrix.append(doc_matrix_row)

100%|██████████| 73/73 [00:00<00:00, 180.62it/s]


In [41]:
article_indexes = pd.DataFrame({
    'article_name': article_names,
    'row_num_in_doc_matrix': range(len(article_names))
})

In [42]:
term_indexes = pd.DataFrame({
    'term': all_term_vector,
    'col_num_in_doc_matrix': range(len(all_term_vector))
})

In [45]:
article_indexes.to_csv('termdocmatrix/article_indexes.csv', index=False)

term_indexes.to_csv('termdocmatrix/term_indexes.csv', index=False)

np.savetxt(
    fname = 'termdocmatrix/doc_matrix.txt',
    X = np.array(doc_matrix).astype(int),
    delimiter = ',',
    newline = '\n',
    fmt = '%u'
)

In [53]:

def save_all_to_output(
    article_indexes_filename: str,
    term_indexes_filename: str,
    doc_matrix_filename: str
):

    article_indexes.to_csv('termdocmatrix/article_indexes.csv', index=False)

    term_indexes.to_csv('termdocmatrix/term_indexes.csv', index=False)

    np.savetxt(
        fname = 'termdocmatrix/doc_matrix.txt',
        X = np.array(doc_matrix).astype(int),
        delimiter = ',',
        newline = '\n',
        fmt = '%u'
    )

In [55]:

def save_all_to_output(

    article_indexes: pd.DataFrame,
    article_indexes_filename: str,

    term_indexes: pd.DataFrame,
    term_indexes_filename: str,

    doc_matrix: [[int]],
    doc_matrix_filename: str

) -> None:

    article_indexes.to_csv('termdocmatrix/article_indexes.csv', index=False)

    term_indexes.to_csv('termdocmatrix/term_indexes.csv', index=False)

    np.savetxt(
        fname = 'termdocmatrix/doc_matrix.txt',
        X = np.array(doc_matrix).astype(int),
        delimiter = ',',
        newline = '\n',
        fmt = '%u'
    )

In [57]:
def foo() -> [int]:
    return [1,2,3]

In [58]:
foo()

[1, 2, 3]

'/Users/mike/jieba'

In [67]:
os.path.join(os.path.abspath('data/'), 'a.txt')

'/Users/mike/jieba/data/a.txt'

In [70]:
def get_articles_from_folder(foldername: str) -> ([str], [str]):

    abs_dir_path = os.path.abspath(foldername)
    article_names = os.listdir(abs_dir_path)
    article_contents = list()
    for article_name in article_names:
        full_path = os.path.join(, article_name)
        file_name = full_path
        file = open(file_name, 'r')
        article_content = file.read()
        file.close()
        article_contents.append(article_content)
    
    return (article_names, article_contents)

SyntaxError: invalid syntax (<ipython-input-70-8b86236ed701>, line 7)

In [71]:

# tuple.1 is article_names, tuple.2 is article_contents
def get_articles_from_folder(foldername: str) -> ([str], [str]):

    abs_dir_path = os.path.abspath(foldername)
    article_names = os.listdir(abs_dir_path)
    article_contents = list()
    for article_name in article_names:
        full_path = os.path.join(abs_dir_path, article_name)
        file_name = full_path
        file = open(file_name, 'r')
        article_content = file.read()
        file.close()
        article_contents.append(article_content)
    
    return (article_names, article_contents)

In [73]:
names, contents = get_articles_from_folder('data')

In [78]:
(1,2,)

(1, 2)

In [79]:
from typing import AnyStr, Callable

In [94]:

def cut_articles(
    article_contents: [str], 
    cutter: Callable[[str], AnyStr]
) -> [[str]]:
    article_words = list()
    for i in tqdm(range(len(article_contents))):
        article_content = article_contents[i]
        words = seg.cut(article_content)
        article_words.append(words)
    
    return article_words

In [89]:
cutter = pkuseg.pkuseg()

In [95]:
my_arts = [
    '你好天安门',
    '我爱北京天安门',
    '美丽的蓝天'
]

In [96]:
my_arts

['你好天安门', '我爱北京天安门', '美丽的蓝天']

In [102]:
words = cut_articles(
    my_arts,
    cutter
)

100%|██████████| 3/3 [00:00<00:00, 2013.27it/s]


[['你好', '天安门'], ['我', '爱', '北京', '天安门'], ['美丽', '的', '蓝天']]

In [101]:

# 统计一个列表中各个元素出现的次数
def counter(l: list) -> dict:
    stats = dict()
    for i in range(len(l)):
        item = l[i]
        if item not in stats:
            stats[item] = 1
        else:
            stats[item] = stats[item] + 1
    
    return stats

# 统计每篇文章的词频
def do_stats(article_words: [[str]]) -> [Dict[str, int]]:
    article_stats = list()
    for i in tqdm(range(len(article_words))):
        article_stats.append(counter(article_words[i]))
    
    return article_stats

In [99]:
from typing import Dict

In [107]:
s = do_stats(words)

100%|██████████| 3/3 [00:00<00:00, 13329.36it/s]


In [109]:

# 统计每一个出现过的单词的出现频数
def make_all_term_stats(article_stats: [dict]) -> Dict[str, int]:
    all_term_stats = dict()
    for i in tqdm(range(len(article_stats))):
        article_stat = article_stats[i]
        for word, count in article_stat.items():
            if word not in all_term_stats:
                all_term_stats[word] = count
            else:
                all_term_stats[word] = all_term_stats[word] + count
    
    return all_term_stats

In [110]:
make_all_term_stats(s)

100%|██████████| 3/3 [00:00<00:00, 15968.16it/s]


{'你好': 1, '天安门': 2, '我': 1, '爱': 1, '北京': 1, '美丽': 1, '的': 1, '蓝天': 1}